# 🔍 Similaridade de Cosseno: Como Funciona a Busca Semântica

## 🎯 Objetivo da Prática

Neste laboratório, você irá:
1. Entender como **calcular a similaridade** entre vetores usando **Similaridade de Cosseno**
2. Implementar uma **busca semântica simples** (mini RAG)
3. Comparar diferentes vetores para encontrar o **mais similar** a uma consulta
4. Compreender o conceito matemático que sustenta bancos vetoriais

## 📖 Conceito Fundamental

### O que é Similaridade de Cosseno?

A **Similaridade de Cosseno** mede o **ângulo** entre dois vetores no espaço. Quanto **menor o ângulo**, mais **similares** são os vetores.

**Fórmula:**
$$
\text{similaridade}(A, B) = \frac{A \cdot B}{\|A\| \times \|B\|}
$$

Onde:
- $A \cdot B$ = produto escalar (dot product) dos vetores
- $\|A\|$ e $\|B\|$ = normas (tamanhos) dos vetores

**Resultado:**
- **1.0** = vetores idênticos (ângulo de 0°)
- **0.0** = vetores perpendiculares (ângulo de 90°)
- **~1.0** = vetores muito similares (ângulo pequeno)

### Por que Cosseno e não Distância Euclidiana?

✅ **Cosseno**: Compara **direção** (semântica), ignora magnitude  
❌ **Euclidiana**: Sensível à magnitude (tamanho), pode distorcer resultados

**Exemplo:**
- "Filhote" e "Cachorrinho gigante" têm direção similar (ambos são animais fofos)
- Euclidiana pode achá-los diferentes se um vetor for 2x maior
- Cosseno reconhece que apontam na **mesma direção** → similares!

---


## 🛠️ Passo 1: Importar Bibliotecas

Vamos importar as ferramentas necessárias para calcular a similaridade de cosseno.

**O que vamos usar:**
- `numpy`: Operações matemáticas com vetores e matrizes
- `norm` (de `numpy.linalg`): Calcula a **norma** (tamanho) de um vetor
  - A norma é como medir o "comprimento" de um vetor no espaço
  - Exemplo: norma de `[3, 4]` = √(3² + 4²) = 5


In [1]:
import numpy as np
from numpy.linalg import norm


## 📊 Passo 2: Criar o "Banco de Dados" Vetorial

Aqui criamos um mini banco de dados com 3 documentos, cada um representado por um vetor de 3 dimensões.

### 🧮 Estrutura dos Vetores

Cada vetor tem **3 coordenadas**: `[Animal, Fofo, Metálico]`

Valores variam de **0.00** (ausência total) a **1.00** (presença máxima).

### 🔍 Análise do Banco:

| Documento | Animal | Fofo | Metálico | Interpretação |
|-----------|--------|------|----------|---------------|
| 🐱 **Gatinho dormindo** | 0.99 | 0.95 | 0.00 | É animal, muito fofo, zero metal |
| 🤖 **Robô industrial** | 0.00 | 0.00 | 0.99 | Não é animal, nada fofo, muito metal |
| 🔨 **Martelo de aço** | 0.00 | 0.00 | 0.95 | Não é animal, nada fofo, quase todo metal |

### 💡 Observe:

- **Gatinho** tem valores altos em **Animal** e **Fofo**, zero em **Metálico**
- **Robô** e **Martelo** têm valores altos em **Metálico**, zero nos outros
- Esses vetores representam as "características semânticas" de cada conceito


In [2]:
# 1. Nosso "Banco de Dados" (Vetores simplificados)
# Formato: [Animal, Fofo, Metálico]
dados = {
    "Gatinho dormindo": np.array([0.99, 0.95, 0.00]),
    "Robô industrial":  np.array([0.00, 0.00, 0.99]),
    "Martelo de aço":   np.array([0.00, 0.00, 0.95])
}


## 🔎 Passo 3: Definir a Consulta (Query) e a Função de Similaridade

Agora vamos criar:
1. O **vetor de consulta** que representa a pesquisa do usuário
2. A **função** que calcula a similaridade de cosseno entre dois vetores

### 🐶 A Consulta: "Filhote de cão"

O usuário pesquisou por **"Filhote de cão"**. Um modelo de embedding converteu isso no vetor:

**Query:** `[0.98, 0.95, 0.00]`

| Característica | Valor | Interpretação |
|----------------|-------|---------------|
| Animal | 0.98 | Sim, é um animal! |
| Fofo | 0.95 | Muito fofo! |
| Metálico | 0.00 | Nada de metal |

**Comparando com o banco:**
- **Gatinho:** `[0.99, 0.95, 0.00]` → Muito similar ao Filhote!
- **Robô:** `[0.00, 0.00, 0.99]` → Completamente diferente
- **Martelo:** `[0.00, 0.00, 0.95]` → Também muito diferente

### 🧮 A Função `calcular_similaridade`

Implementa a fórmula da similaridade de cosseno:

$$
\text{similaridade}(A, B) = \frac{A \cdot B}{\|A\| \times \|B\|}
$$

**Passo a passo:**
1. `np.dot(vetor_a, vetor_b)`: Calcula o **produto escalar** (multiplica coordenadas correspondentes e soma)
2. `norm(vetor_a)` e `norm(vetor_b)`: Calcula as **normas** (tamanhos) dos vetores
3. Divide o produto escalar pelas normas → resultado entre 0 e 1

**Exemplo manual:**
- Query = `[0.98, 0.95, 0.00]`, Gatinho = `[0.99, 0.95, 0.00]`
- Produto escalar = (0.98×0.99) + (0.95×0.95) + (0.00×0.00) = 0.9702 + 0.9025 = 1.8727
- Norma Query = √(0.98² + 0.95²) ≈ 1.365
- Norma Gatinho = √(0.99² + 0.95²) ≈ 1.371
- **Similaridade = 1.8727 / (1.365 × 1.371) ≈ 1.000** (quase perfeito!)


In [6]:
# 2. A Pesquisa do Usuário ("Filhote de cão")
# A IA converteu a pesquisa para este vetor:
query_vetor = np.array([0.98, 0.95, 0.00])

def calcular_similaridade(vetor_a, vetor_b):
    # Cálculo da Similaridade de Cosseno (vai de 0 a 1)
    # `norm` calcula a norma de um vetor ou matriz, ou seja, uma medida do seu "tamanho" ou "comprimento"
    cos_sim = np.dot(vetor_a, vetor_b) / (norm(vetor_a) * norm(vetor_b))
    return cos_sim


## 🚀 Passo 4: Executar a Busca Semântica

Agora vamos **buscar** no banco de dados qual documento é **mais similar** à consulta "Filhote de cão".

### 🔄 Como funciona o algoritmo:

1. **Para cada documento** no banco:
   - Calcula a similaridade entre o vetor da query e o vetor do documento
   - Armazena o resultado (texto + score)

2. **Ordena** os resultados do **maior** para o **menor** score
   - Score próximo de 1.0 = muito similar
   - Score próximo de 0.0 = pouco similar

3. **Exibe** os resultados ordenados

### 📊 Resultados Esperados:

Baseando-nos nas características dos vetores:

| Rank | Documento | Score Esperado | Motivo |
|------|-----------|----------------|--------|
| 🥇 1º | Gatinho dormindo | **~1.000** | Quase idêntico ao Filhote (ambos animais fofos) |
| 🥈 2º | Martelo de aço | **~0.000** | Completamente diferente (metal vs. animal) |
| 🥉 3º | Robô industrial | **~0.000** | Também muito diferente (metal vs. animal) |

**Por que Gatinho vence?**
- Query: `[0.98, 0.95, 0.00]` (animal fofo)
- Gatinho: `[0.99, 0.95, 0.00]` (animal fofo)
- Ambos têm **valores altos** nas mesmas dimensões → **direção similar** no espaço!

### 💡 Conexão com RAG:

Isso é exatamente o que acontece em sistemas RAG:
1. Usuário faz uma pergunta
2. Pergunta é convertida em vetor (embedding)
3. Sistema busca documentos com vetores **similares**
4. Documentos mais relevantes são retornados


In [7]:

# 3. Executando a Busca
print(f"Pesquisa: 'Filhote de cão' {query_vetor}\n")
print("--- Resultados ---")

resultados = []
for texto, vetor_db in dados.items():
    score = calcular_similaridade(query_vetor, vetor_db)
    resultados.append((texto, score))

# Ordenar do maior score para o menor
resultados.sort(key=lambda x: x[1], reverse=True)

for texto, score in resultados:
    print(f"Texto: '{texto}' | Similaridade: {score:.4f}")

Pesquisa: 'Filhote de cão' [0.98 0.95 0.  ]

--- Resultados ---
Texto: 'Gatinho dormindo' | Similaridade: 1.0000
Texto: 'Robô industrial' | Similaridade: 0.0000
Texto: 'Martelo de aço' | Similaridade: 0.0000


---

## 🎓 Conclusões e Aprendizados

### ✅ O que você aprendeu neste laboratório:

1. **Similaridade de Cosseno é a métrica padrão para busca semântica**
   - Mede o **ângulo** entre vetores (não a distância)
   - Resultado entre 0 (diferentes) e 1 (idênticos)
   - Ignora magnitude, foca em **direção semântica**

2. **Busca semântica é simples conceitualmente**
   - Converter texto em vetor (embedding)
   - Calcular similaridade com todos os vetores do banco
   - Ordenar por score e retornar os mais similares

3. **O algoritmo funciona porque:**
   - Textos similares → embeddings com **direção similar** no espaço
   - Similaridade de cosseno captura essa semelhança
   - É escalável para milhões de documentos (com otimizações)


### 📊 Comparação: Cosseno vs. Euclidiana

| Métrica | Foco | Sensível a Magnitude | Melhor Para |
|---------|------|----------------------|-------------|
| **Cosseno** | Direção (ângulo) | ❌ Não | Busca semântica, texto |
| **Euclidiana** | Distância absoluta | ✅ Sim | Dados numéricos, imagens |

**Exemplo prático:**
- Textos: "gato" vs. "felino gigante"
  - Euclidiana: diferentes (magnitudes diferentes)
  - Cosseno: similares (mesma direção semântica) ✅

### 🧪 Desafios Práticos

**Desafio 1:** Adicione novos documentos ao banco e execute a busca:
```python
# Experimente:
dados["Pássaro cantando"] = np.array([0.95, 0.80, 0.00])  # Animal fofo, não metal
dados["Tesoura de metal"] = np.array([0.00, 0.00, 0.85])  # Não animal, metal

# Execute a busca novamente. Qual será a nova ordem?
```

**Desafio 2:** Teste com uma query diferente:
```python
# Query: "Objeto de metal"
query_vetor = np.array([0.00, 0.00, 0.98])

# Qual documento terá o maior score agora?
# Dica: Robô e Martelo devem ficar em 1º e 2º
```

**Desafio 3:** Calcule manualmente e compare:
```python
# Pegue 2 vetores:
A = np.array([0.99, 0.95, 0.00])  # Gatinho
B = np.array([0.00, 0.00, 0.99])  # Robô

# Calcule à mão:
# 1. Produto escalar: (0.99×0) + (0.95×0) + (0×0.99) = 0
# 2. Normas: sqrt(0.99² + 0.95²) × sqrt(0.99²) = 1.37 × 0.99
# 3. Similaridade: 0 / (1.37 × 0.99) = 0.0
# Confirme com a função!
```

**Desafio 4:** Entenda o impacto da normalização:
```python
# Teste com vetores não normalizados
vetor_grande = np.array([9.9, 9.5, 0.0])   # 10x maior que Gatinho
vetor_pequeno = np.array([0.099, 0.095, 0.0])  # 10x menor

# A similaridade entre eles será próxima de 1.0?
# Por quê? (Dica: cosseno ignora magnitude!)
```

---

**Parabéns! 🎉** Você implementou sua primeira busca semântica e entendeu o coração dos bancos de dados vetoriais!